In [80]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [81]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    nav.get('https://www.google.com/')
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

    elementos = nav.find_element('class name', 'GKS7s').click()

    lista_resultados = nav.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        # print(nome)

        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_produto, nome)
        
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            try:
                preco = float(preco)
            except:
                continue

            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    nav.get('https://www.buscape.com.br/')
    nav.find_element('class name', 'AutoCompleteStyle_input__WAC2Y').send_keys(produto, Keys.ENTER)
    
    time.sleep(5)
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute('href')

        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_produto, nome)
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            try:
                preco = float(preco)
            except:
                continue
            if preco_minimo <= preco <= preco_maximo:

                lista_ofertas.append((nome, preco, link))

    return lista_ofertas

In [82]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:    

    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None

    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
display(tabela_ofertas)

,produto,preco,link
0,"apple iphone 12, 64 gb - preto",3279.00,https://www.google.com/url?url=https://www.ama...
1,usado: iphone 12 64gb azul excelente - trocafone,3049.00,https://www.google.com/url?url=https://www.pon...
2,usado: iphone 12 64gb branco excelente - troca...,3049.00,https://www.google.com/url?url=https://www.cas...
3,usado: iphone 12 64gb preto excelente - trocafone,3049.00,https://www.google.com/url?url=https://www.pon...
4,apple celular | iphone 12 64gb | usado | taman...,3115.00,https://www.google.com/url?url=https://www.enj...
5,"iphone 12 64gb- branco, , color branco",3260.00,https://www.google.com/url?url=https://www.tra...
6,iphone 12 apple 64gb tela de 6.1 polegadas câm...,3482.28,https://www.google.com/url?url=https://www.leb...
7,apple iphone 12 (64gb) preto - vitrine original,3400.00,https://www.google.com/url?url=https://www.wor...
0,pc gamer intel i5 16gb rtx 3060 12gb hd 1tb ss...,4159.00,https://www.google.com/url?url=https://www.2el...
0,gigabyte placa gráfica geforce rtx 3060 vision...,4488.88,https://www.buscape.com.br/lead?oid=550418315&...


In [83]:
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

In [85]:
import win32com.client as win32

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'email'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f'''
    <p>Prezados</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att., Matheus</p>

'''
    mail.Send()

In [ ]:
nav.quit()